In [1]:
def real_diarize():
    """"""
    # VAD
    waveform, _ = read_audio("./mono_file.wav")
    vad_service = VadService()
    speech_ts, _ = vad_service(waveform, False)

    # Segmentation

    # Clustering

    # Scoring

## VAD

In [12]:
import json

from nemo.collections.asr.models.label_models import EncDecSpeakerLabelModel
from nemo.collections.asr.models.msdd_models import ClusteringDiarizer

from omegaconf import OmegaConf


# Load yaml file
with open("./config/nemo/diar_infer_telephonic.yaml") as f:
    cfg = OmegaConf.load(f)

meta = {
    "audio_filepath":  "mono_file.wav",
    "offset": 0,
    "duration": None,
    "label": "infer",
    "text": "-",
    "rttm_filepath": None,
    "uem_filepath": None,
}

manifest_path = "infer_manifest.json"
with open("infer_manifest.json", "w") as fp:
    json.dump(meta, fp)
    fp.write("\n")

cfg.diarizer.manifest_filepath = str(manifest_path)
cfg.diarizer.out_dir = "infer_out_dir"

speaker_model = EncDecSpeakerLabelModel.from_pretrained(
    model_name="titanet_large", map_location=None
)
speaker_params = {
    "window_length_in_sec": [1.5, 1.25, 1.0, 0.75, 0.5],
    "shift_length_in_sec": [0.75, 0.625, 0.5, 0.375, 0.25],
    "multiscale_weights": [1, 1, 1, 1, 1],
    "save_embeddings": True,
}
cluster_params = {
    "oracle_num_speakers": False,
    "max_num_speakers": 8,
    "enhanced_count_thres": 80,
    "max_rp_threshold": 0.25,
    "sparse_search_volume": 30,
    "maj_vote_spk_count": False,
}

clus_diar_model = ClusteringDiarizer(cfg=cfg, speaker_model=speaker_model)

[NeMo I 2023-07-25 09:06:34 cloud:58] Found existing object /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-07-25 09:06:34 cloud:64] Re-using file from: /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo
[NeMo I 2023-07-25 09:06:34 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-25 09:06:35 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-07-25 09:06:35 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-07-25 09:06:35 features:291] PADDING: 16
[NeMo I 2023-07-25 09:06:35 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/titanet-l/11ba0924fdf87c049e339adbf6899d48/titanet-l.nemo.
[NeMo I 2023-07-25 09:06:35 clustering_diarizer:127] Loading pretrained vad_multilingual_marblenet model from NGC
[NeMo I 2023-07-25 09:06:35 cloud:58] Found existing object /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.
[NeMo I 2023-07-25 09:06:35 cloud:64] Re-using file from: /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo
[NeMo I 2023-07-25 09:06:35 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-25 09:06:35 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/ami_train_0.63.json,/manifests/freesound_background_train.json,/manifests/freesound_laughter_train.json,/manifests/fisher_2004_background.json,/manifests/fisher_2004_speech_sampled.json,/manifests/google_train_manifest.json,/manifests/icsi_all_0.63.json,/manifests/musan_freesound_train.json,/manifests/musan_music_train.json,/manifests/musan_soundbible_train.json,/manifests/mandarin_train_sample.json,/manifests/german_train_sample.json,/manifests/spanish_train_sample.json,/manifests/french_train_sample.json,/manifests/russian_train_sample.json
    sample_rate: 16000
    labels:
    - background
    - speech
    batch_size: 256
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: sca

[NeMo I 2023-07-25 09:06:35 features:291] PADDING: 16
[NeMo I 2023-07-25 09:06:35 save_restore_connector:249] Model EncDecClassificationModel was successfully restored from /home/chainyo/.cache/torch/NeMo/NeMo_1.19.1/vad_multilingual_marblenet/670f425c7f186060b7a7268ba6dfacb2/vad_multilingual_marblenet.nemo.


In [16]:
import librosa
import soundfile as sf

filepath = "./mono_file.mp3"
waveform, sample_rate = librosa.load(filepath, sr=None)
sf.write("./mono_file.wav", waveform, sample_rate, "PCM_16")

In [17]:
clus_diar_model.diarize()

[NeMo W 2023-07-25 09:37:44 clustering_diarizer:411] Deleting previous clustering diarizer outputs.


[NeMo I 2023-07-25 09:37:44 speaker_utils:93] Number of files to diarize: 1
[NeMo I 2023-07-25 09:37:44 clustering_diarizer:309] Split long audio file to avoid CUDA memory issue


splitting manifest: 100%|██████████| 1/1 [00:00<00:00, 11.75it/s]

[NeMo I 2023-07-25 09:37:44 vad_utils:101] The prepared manifest file exists. Overwriting!
[NeMo I 2023-07-25 09:37:44 classification_models:268] Perform streaming frame-level VAD
[NeMo I 2023-07-25 09:37:44 collections:298] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-07-25 09:37:44 collections:299] Dataset loaded with 3 items, total duration of  0.04 hours.
[NeMo I 2023-07-25 09:37:44 collections:301] # 3 files loaded accounting to # 1 labels


[NeMo I 2023-07-25 09:37:46 clustering_diarizer:250] Generating predictions with overlapping input segments


[NeMo I 2023-07-25 09:37:47 clustering_diarizer:262] Converting frame level prediction to speech/no-speech segment in start and end times format.


creating speech segments: 100%|██████████| 1/1 [00:00<00:00,  3.86it/s]

[NeMo I 2023-07-25 09:37:47 clustering_diarizer:287] Subsegmentation for embedding extraction: scale0, infer_out_dir/speaker_outputs/subsegments_scale0.json
[NeMo I 2023-07-25 09:37:47 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2023-07-25 09:37:47 collections:298] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-07-25 09:37:47 collections:299] Dataset loaded with 104 items, total duration of  0.04 hours.
[NeMo I 2023-07-25 09:37:47 collections:301] # 104 files loaded accounting to # 1 labels


[NeMo I 2023-07-25 09:37:48 clustering_diarizer:389] Saved embedding files to infer_out_dir/speaker_outputs/embeddings
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:287] Subsegmentation for embedding extraction: scale1, infer_out_dir/speaker_outputs/subsegments_scale1.json
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:343] Extracting embeddings for Diarization
[NeMo I 2023-07-25 09:37:48 collections:298] Filtered duration for loading collection is  0.00 hours.
[NeMo I 2023-07-25 09:37:48 collections:299] Dataset loaded with 132 items, total duration of  0.04 hours.
[NeMo I 2023-07-25 09:37:48 collections:301] # 132 files loaded accounting to # 1 labels
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:389] Saved embedding files to infer_out_dir/speaker_outputs/embeddings
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:287] Subsegmentation for embedding extraction: scale2, infer_out_dir/speaker_outputs/subsegments_scale2.json
[NeMo I 2023-07-25 09:37:48 clustering_diarizer:343] Extrac

In [1]:
from typing import List, Optional, Tuple, Union

import torch
import torchaudio
from faster_whisper.vad import VadOptions, get_speech_timestamps


class VadService:
    """VAD Service for audio files."""

    def __init__(self) -> None:
        """Initialize the VAD Service."""
        self.sample_rate = 16000
        self.options = VadOptions(
            threshold=0.5,
            min_speech_duration_ms=250,
            max_speech_duration_s=30,
            min_silence_duration_ms=100,
            window_size_samples=512,
            speech_pad_ms=30,
        )

    def __call__(
        self, waveform: torch.Tensor, group_timestamps: Optional[bool] = True
    ) -> Tuple[Union[List[dict], List[List[dict]]], torch.Tensor]:
        """
        Use the VAD model to get the speech timestamps. Dual channel pipeline.

        Args:
            waveform (torch.Tensor): Audio tensor.
            group_timestamps (Optional[bool], optional): Group timestamps. Defaults to True.

        Returns:
            Tuple[Union[List[dict], List[List[dict]]], torch.Tensor]: Speech timestamps and audio tensor.
        """
        if waveform.size(0) == 1:
            waveform = waveform.squeeze(0)

        speech_timestamps = get_speech_timestamps(
            audio=waveform, vad_options=self.options
        )

        _speech_timestamps_list = [
            {"start": ts["start"], "end": ts["end"]} for ts in speech_timestamps
        ]

        if group_timestamps:
            speech_timestamps_list = self.group_timestamps(_speech_timestamps_list)
        else:
            speech_timestamps_list = _speech_timestamps_list

        return speech_timestamps_list, waveform

    def group_timestamps(
        self, timestamps: List[dict], threshold: Optional[float] = 3.0
    ) -> List[List[dict]]:
        """
        Group timestamps based on a threshold.

        Args:
            timestamps (List[dict]): List of timestamps.
            threshold (float, optional): Threshold to use for grouping. Defaults to 3.0.

        Returns:
            List[List[dict]]: List of grouped timestamps.
        """
        grouped_segments = [[]]

        for i in range(len(timestamps)):
            if (
                i > 0
                and (timestamps[i]["start"] - timestamps[i - 1]["end"]) > threshold
            ):
                grouped_segments.append([])

            grouped_segments[-1].append(timestamps[i])

        return grouped_segments

    def save_audio(self, filepath: str, audio: torch.Tensor) -> None:
        """
        Save audio tensor to file.

        Args:
            filepath (str): Path to save the audio file.
            audio (torch.Tensor): Audio tensor.
        """
        torchaudio.save(
            filepath, audio.unsqueeze(0), self.sample_rate, bits_per_sample=16
        )

def read_audio(filepath: str, sample_rate: int = 16000) -> Tuple[torch.Tensor, float]:
    """
    Read an audio file and return the audio tensor.

    Args:
        filepath (str): Path to the audio file.
        sample_rate (int): The sample rate of the audio file. Defaults to 16000.

    Returns:
        Tuple[torch.Tensor, float]: The audio tensor and the audio duration.
    """
    wav, sr = torchaudio.load(filepath)

    if wav.size(0) > 1:
        wav = wav.mean(dim=0, keepdim=True)

    if sr != sample_rate:
        transform = torchaudio.transforms.Resample(orig_freq=sr, new_freq=sample_rate)
        wav = transform(wav)
        sr = sample_rate

    audio_duration = float(wav.shape[1]) / sample_rate

    return wav.squeeze(0), audio_duration

In [2]:
def sr2s(v: int) -> float:
    """
    Convert milliseconds to seconds.

    Args:
        v (int): Value in milliseconds.

    Returns:
        float: Value in seconds.
    """
    return v / 16000

In [51]:
waveform, _ = read_audio("./mono_file.wav")

vad_service = VadService()

speech_ts, _ = vad_service(waveform, True)
for ts in speech_ts:
    _ts = ts[0]
    print(
        f"Start: {sr2s(_ts['start'])}, End: {sr2s(_ts['end'])}"
    )

42
Start: 12.514, End: 12.99
Start: 13.25, End: 14.366
Start: 15.042, End: 16.862
Start: 17.762, End: 18.814
Start: 19.426, End: 20.766
Start: 21.666, End: 24.83
Start: 26.178, End: 29.886
Start: 30.786, End: 33.022
Start: 34.146, End: 37.214
Start: 38.338, End: 40.318
Start: 41.218, End: 42.782
Start: 43.682, End: 44.318
Start: 45.73, End: 46.494
Start: 47.778, End: 50.366
Start: 51.106, End: 52.926
Start: 53.954, End: 55.582
Start: 55.682, End: 56.926
Start: 57.954, End: 60.286
Start: 61.154, End: 64.254
Start: 65.026, End: 67.614
Start: 68.418, End: 68.99
Start: 69.922, End: 71.55
Start: 72.578, End: 75.838
Start: 76.61, End: 77.918
Start: 78.562, End: 79.454
Start: 79.746, End: 81.086
Start: 82.05, End: 83.902
Start: 84.738, End: 86.462
Start: 87.586, End: 90.782
Start: 91.746, End: 96.542
Start: 97.73, End: 98.27
Start: 99.586, End: 100.03
Start: 100.162, End: 100.862
Start: 101.794, End: 103.454
Start: 104.898, End: 107.486
Start: 108.226, End: 109.854
Start: 114.274, End: 114.75

## Segmentation

In [3]:
window_length_in_sec = [1.5, 1.25, 1.0, 0.75, 0.5] # Window length(s) in sec (floating-point number). either a number or a list. ex) 1.5 or [1.5,1.0,0.5]
shift_length_in_sec = [0.75, 0.625, 0.5, 0.375, 0.25] # Shift length(s) in sec (floating-point number). either a number or a list. ex) 0.75 or [0.75,0.5,0.25]
multiscale_weights = [1, 1, 1, 1, 1]

In [4]:
window_lengths_in_sec = window_length_in_sec
shift_lengths_in_sec = shift_length_in_sec

window_lengths, shift_lengths, multiscale_weights = (
    window_lengths_in_sec,
    shift_lengths_in_sec,
    multiscale_weights,
)

multiscale_args_dict = {'use_single_scale_clustering': False}
scale_dict = {k: (w, s) for k, (w, s) in enumerate(zip(window_lengths, shift_lengths))}

In [5]:
import math

def get_subsegments(segment_start: float, segment_end: float, window: float, shift: float) -> List[List[float]]:
    """
    Return a list of subsegments based on the segment start and end time and the window and shift length.

    Args:
        segment_start (float): Segment start time.
        segment_end (float): Segment end time.
        window (float): Window length.
        shift (float): Shift length.

    Returns:
        List[List[float]]: List of subsegments with start time and duration.
    """
    start = segment_start
    duration = segment_end - segment_start
    base = math.ceil((duration - window) / shift)
    
    subsegments: List[List[float]] = []
    slices = 1 if base < 0 else base + 1
    for slice_id in range(slices):
        end = start + window

        if end > segment_end:
            end = segment_end

        subsegments.append([start, end - start])

        start = segment_start + (slice_id + 1) * shift

    return subsegments

In [7]:
def _run_segmentation(
    vad_outputs: List[dict],
    window: float,
    shift: float,
    min_subsegment_duration: float = 0.05,
) -> List[dict]:
    """"""
    scale_segment = []
    for segment in vad_outputs:
        segment_start, segment_end = sr2s(segment["start"]), sr2s(segment["end"])
        subsegments = get_subsegments(segment_start, segment_end, window, shift)

        for subsegment in subsegments:
            start, duration = subsegment
            if duration > min_subsegment_duration:
                scale_segment.append({"offset": start, "duration": duration})

    return scale_segment

In [ ]:
from nemo.collections.asr.models.msdd_models import EncDecSpeakerLabelModel

speaker_model = EncDecSpeakerLabelModel.from_pretrained(
    model_name="titanet_large", map_location=None
)

def _extract_embeddings(scale_segments: List[dict], scale_idx: int, num_scales: int):
    """
    This method extracts speaker embeddings from segments passed through manifest_file
    Optionally you may save the intermediate speaker embeddings for debugging or any use. 
    """

    # self._setup_spkr_test_data(manifest_file)
    # self.embeddings = {}
    # self._speaker_model.eval()
    # self.time_stamps = {}

    # all_embs = torch.empty([0])
    # for test_batch in tqdm(
    #     self._speaker_model.test_dataloader(),
    #     desc=f'[{scale_idx+1}/{num_scales}] extract embeddings',
    #     leave=True,
    #     disable=not self.verbose,
    # ):
    #     test_batch = [x.to(self._speaker_model.device) for x in test_batch]
    #     audio_signal, audio_signal_len, labels, slices = test_batch
    #     with autocast():
    #         _, embs = self._speaker_model.forward(input_signal=audio_signal, input_signal_length=audio_signal_len)
    #         emb_shape = embs.shape[-1]
    #         embs = embs.view(-1, emb_shape)
    #         all_embs = torch.cat((all_embs, embs.cpu().detach()), dim=0)
    #     del test_batch

    embeddings, time_stamps = [], []
    for i, segment in enumerate(scale_segments):
        if i == 0:
            embeddings = all_embs[i].view(1, -1)
        else:
            embeddings = torch.cat((embeddings, all_embs[i].view(1, -1)))

        time_stamps.append([segment['offset'], segment['duration']])
        
        # uniq_name = get_uniqname_from_filepath(dic['audio_filepath'])
        # if uniq_name in self.embeddings:
        #     self.embeddings[uniq_name] = torch.cat((self.embeddings[uniq_name], all_embs[i].view(1, -1)))
        # else:
        #     self.embeddings[uniq_name] = all_embs[i].view(1, -1)
        # if uniq_name not in self.time_stamps:
        #     self.time_stamps[uniq_name] = []
        
        # start = dic['offset']
        # end = start + dic['duration']
        # self.time_stamps[uniq_name].append([start, end])

    if self._speaker_params.save_embeddings:
        embedding_dir = os.path.join(self._speaker_dir, 'embeddings')
        if not os.path.exists(embedding_dir):
            os.makedirs(embedding_dir, exist_ok=True)

        prefix = get_uniqname_from_filepath(manifest_file)
        name = os.path.join(embedding_dir, prefix)
        self._embeddings_file = name + f'_embeddings.pkl'
        pkl.dump(self.embeddings, open(self._embeddings_file, 'wb'))
        logging.info("Saved embedding files to {}".format(embedding_dir))

In [8]:
waveform, _ = read_audio("./mono_file.wav")
vad_service = VadService()

vad_outputs, _ = vad_service(waveform, False)

scales = scale_dict.items()
for scale_idx, (window, shift) in scales:
    print(scale_idx, window, shift)
    # Segmentation for the current scale (scale_idx)
    scale_segments = _run_segmentation(vad_outputs, window, shift)

    # Embedding Extraction for the current scale (scale_idx)
    embeddings = _extract_embeddings(scale_segments, scale_idx, len(scales))

    # self.multiscale_embeddings_and_timestamps[scale_idx] = [self.embeddings, self.time_stamps]

0 1.5 0.75
[{'offset': 12.514, 'duration': 0.47600000000000087}, {'offset': 13.25, 'duration': 1.1159999999999997}, {'offset': 15.042, 'duration': 1.5000000000000018}, {'offset': 15.792, 'duration': 1.0699999999999985}, {'offset': 17.762, 'duration': 1.0519999999999996}, {'offset': 19.426, 'duration': 1.3399999999999999}, {'offset': 21.666, 'duration': 1.5}, {'offset': 22.416, 'duration': 1.5}, {'offset': 23.166, 'duration': 1.5}, {'offset': 23.916, 'duration': 0.9139999999999979}, {'offset': 26.178, 'duration': 1.5}, {'offset': 26.928, 'duration': 1.5}, {'offset': 27.678, 'duration': 1.5}, {'offset': 28.428, 'duration': 1.4579999999999984}, {'offset': 30.786, 'duration': 1.5}, {'offset': 31.536, 'duration': 1.485999999999997}, {'offset': 34.146, 'duration': 1.5}, {'offset': 34.896, 'duration': 1.5}, {'offset': 35.646, 'duration': 1.5}, {'offset': 36.396, 'duration': 0.8179999999999978}, {'offset': 38.338, 'duration': 1.5}, {'offset': 39.088, 'duration': 1.2299999999999969}, {'offset':